In [1]:
import numpy as np, h5py

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_norm') # movie for training
frhist = datafile.get('FRhist_tr') # firing rate histograms

In [2]:
import keras
from keras.layers import TimeDistributed, Dense

Using TensorFlow backend.


In [3]:
def run_model(hidden1, hidden2):
    model = keras.models.Sequential()
    model.add(TimeDistributed(Dense(hidden1, activation='sigmoid'), input_shape=movie.shape[1:]))
    model.add(TimeDistributed(Dense(hidden2, activation='sigmoid')))
    model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
    history = model.fit(movie, frhist, epochs=40, batch_size=32, validation_split=0.2, shuffle=True)

# for hidden in [30, 40, 50, 60, 75, 100, 150]:
#     run_model(hidden, hidden)

for hidden in [65, 75, 85, 95]:
    run_model(hidden, hidden)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 150, 65)           798785    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 150, 65)           4290      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 150, 54)           3564      
Total params: 806,639
Trainable params: 806,639
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/40
230/230 [==============================] - 4s 18ms/step - loss: 0.7240 - val_loss: 0.6505
Epoch 2/40
230/230 [==============================] - 3s 15ms/step - loss: 0.6122 - val_loss: 0.5614
Epoch 3/40
230/230 [==============================] - 3s 14ms/step - loss: 0.5321 - val_loss: 0.4943
Epoch 4/40
230/230 [==============================] 

230/230 [==============================] - 3s 14ms/step - loss: 0.3438 - val_loss: 0.3443
Epoch 12/40
230/230 [==============================] - 3s 15ms/step - loss: 0.3433 - val_loss: 0.3439
Epoch 13/40
230/230 [==============================] - 3s 14ms/step - loss: 0.3428 - val_loss: 0.3436
Epoch 14/40
230/230 [==============================] - 3s 14ms/step - loss: 0.3425 - val_loss: 0.3433
Epoch 15/40
230/230 [==============================] - 3s 15ms/step - loss: 0.3423 - val_loss: 0.3432
Epoch 16/40
230/230 [==============================] - 4s 16ms/step - loss: 0.3420 - val_loss: 0.3430
Epoch 17/40
230/230 [==============================] - 4s 16ms/step - loss: 0.3419 - val_loss: 0.3430
Epoch 18/40
230/230 [==============================] - 4s 16ms/step - loss: 0.3418 - val_loss: 0.3428
Epoch 19/40
230/230 [==============================] - 4s 16ms/step - loss: 0.3415 - val_loss: 0.3427
Epoch 20/40
230/230 [==============================] - 4s 16ms/step - loss: 0.3414 - val_loss:

In [6]:
# why does this overfit more than the sigmoid version?

model = keras.models.Sequential()
model.add(TimeDistributed(Dense(75, activation='tanh'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(75, activation='tanh')))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
history = model.fit(movie, frhist, epochs=40, batch_size=32, validation_split=0.2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 150, 75)           921675    
_________________________________________________________________
time_distributed_17 (TimeDis (None, 150, 75)           5700      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 150, 54)           4104      
Total params: 931,479
Trainable params: 931,479
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/40
230/230 [==============================] - 4s 17ms/step - loss: 0.6450 - val_loss: 0.5318
Epoch 2/40
230/230 [==============================] - 4s 16ms/step - loss: 0.4914 - val_loss: 0.4347
Epoch 3/40
230/230 [==============================] - 4s 16ms/step - loss: 0.4120 - val_loss: 0.3830
Epoch 4/40
230/230 [==============================] 

In [9]:
# recurrence doesn't improve accuracy!
from keras.layers import CuDNNLSTM, Dense
from keras.layers import Conv2D, MaxPooling2D, Dropout, Reshape
# from keras.layers import TimeDistributed

model = keras.models.Sequential()

model.add(CuDNNLSTM(90, return_sequences=True, input_shape=movie.shape[1:]))
model.add(Dense(75, activation='sigmoid'))
model.add(Dense(frhist.shape[2], activation='softplus'))
adamopt = keras.optimizers.Adam(lr=0.001, decay=1e-7)

model.compile(optimizer=adamopt, loss='poisson')
model.summary()

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 150, 90)           4456800   
_________________________________________________________________
dense_23 (Dense)             (None, 150, 75)           6825      
_________________________________________________________________
dense_24 (Dense)             (None, 150, 54)           4104      
Total params: 4,467,729
Trainable params: 4,467,729
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 4s 18ms/step - loss: 0.6921 - val_loss: 0.6045
Epoch 2/200
230/230 [==============================] - 4s 17ms/step - loss: 0.5633 - val_loss: 0.5062
Epoch 3/200
230/230 [==============================] - 4s 17ms/step - loss: 0.4770 - val_loss: 0.4369
Epoch 4/200
230/230 [========================

In [10]:
# bottom-heavy
model = keras.models.Sequential()
model.add(TimeDistributed(Dense(100, activation='sigmoid'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(60, activation='sigmoid')))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
history = model.fit(movie, frhist, epochs=40, batch_size=32, validation_split=0.2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 150, 100)          1228900   
_________________________________________________________________
time_distributed_20 (TimeDis (None, 150, 60)           6060      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 150, 54)           3294      
Total params: 1,238,254
Trainable params: 1,238,254
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/40
230/230 [==============================] - 4s 18ms/step - loss: 0.7269 - val_loss: 0.6530
Epoch 2/40
230/230 [==============================] - 3s 15ms/step - loss: 0.6192 - val_loss: 0.5711
Epoch 3/40
230/230 [==============================] - 4s 15ms/step - loss: 0.5451 - val_loss: 0.5085
Epoch 4/40
230/230 [============================

In [11]:
# top-heavy -- converged faster but didn't reach as high of accuracy
model = keras.models.Sequential()
model.add(TimeDistributed(Dense(60, activation='sigmoid'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(100, activation='sigmoid')))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
history = model.fit(movie, frhist, epochs=40, batch_size=32, validation_split=0.2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_22 (TimeDis (None, 150, 60)           737340    
_________________________________________________________________
time_distributed_23 (TimeDis (None, 150, 100)          6100      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 150, 54)           5454      
Total params: 748,894
Trainable params: 748,894
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/40
230/230 [==============================] - 4s 16ms/step - loss: 0.7221 - val_loss: 0.6271
Epoch 2/40
230/230 [==============================] - 3s 14ms/step - loss: 0.5798 - val_loss: 0.5176
Epoch 3/40
230/230 [==============================] - 3s 15ms/step - loss: 0.4847 - val_loss: 0.4437
Epoch 4/40
230/230 [==============================] 

Looks like a dense network with two 75-neuron hidden layers is the best option.

In [17]:
model = keras.models.Sequential()
model.add(TimeDistributed(Dense(50, activation='sigmoid'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(65, activation='sigmoid')))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_39 (TimeDis (None, 150, 50)           614450    
_________________________________________________________________
time_distributed_40 (TimeDis (None, 150, 65)           3315      
_________________________________________________________________
time_distributed_41 (TimeDis (None, 150, 54)           3564      
Total params: 621,329
Trainable params: 621,329
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 4s 18ms/step - loss: 0.7099 - val_loss: 0.6433
Epoch 2/200
230/230 [==============================] - 4s 16ms/step - loss: 0.6095 - val_loss: 0.5620
Epoch 3/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5354 - val_loss: 0.4982
Epoch 4/200
230/230 [============================

In [3]:
model = keras.models.Sequential()
model.add(TimeDistributed(Dense(50, activation='sigmoid'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 150, 50)           614450    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 150, 54)           2754      
Total params: 617,204
Trainable params: 617,204
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 4s 19ms/step - loss: 0.6763 - val_loss: 0.6240
Epoch 2/200
230/230 [==============================] - 4s 16ms/step - loss: 0.6100 - val_loss: 0.5918
Epoch 3/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5780 - val_loss: 0.5621
Epoch 4/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5491 - val_loss: 0.5350
Epoch 5/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5

In [ ]:
# single hidden-layer models do almost as well
def run_model(hidden):
    model = keras.models.Sequential()
    model.add(TimeDistributed(Dense(hidden, activation='sigmoid'), input_shape=movie.shape[1:]))
    model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])
    
for h in [80, 70, 60]:
    run_model(h)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_9 (TimeDist (None, 150, 80)           983120    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 150, 54)           4374      
Total params: 987,494
Trainable params: 987,494
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 4s 17ms/step - loss: 0.6330 - val_loss: 0.5450
Epoch 2/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5177 - val_loss: 0.4816
Epoch 3/200
230/230 [==============================] - 4s 16ms/step - loss: 0.4611 - val_loss: 0.4366
Epoch 4/200
230/230 [==============================] - 4s 16ms/step - loss: 0.4220 - val_loss: 0.4062
Epoch 5/200
230/230 [==============================] - 4s 16ms/step - loss: 0.3